In [ ]:
import random

"""
movies_list = []
movies_similarity = []
friends = {1:[]}
friends_watched = {user_id: [movie_id, ...]}
"""

movies_list = range(1, 100)
movies_similarity = []

while len(movies_similarity) < 31:
    random_movie = random.sample(movies_list, k=2)
    if random_movie not in movies_similarity:
        movies_similarity.append(tuple(random_movie))

# print(movies_similarity)

users = range(1,30)

friends = dict()

for user_id in users:
    
    friends_num = random.randint(0, 9)
    friends_sample = random.sample(users, k=friends_num)

    if user_id in friends_sample:
        friends_sample.remove(user_id)

    friends[user_id] = friends_sample

# for i, j in friends.items():
#     if i in j:
#         print(False)
#         break

friends_watched = dict()

for user_id in users:

    watched_num = random.randint(0,20)
    friends_watched[user_id] = random.sample(movies_list, k=watched_num)

# print(friends_watched)

def discussability(movies_list, movies_similarity, friends, friends_watched):
    pass


def uniqueness(movies_list, movies_similarity, friends, friends_watched):
    pass